In [1]:
import torch

In [2]:
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
else:
    print("CUDA is not available.")

GPU 0: AMD Radeon RX 9070 XT
GPU 1: AMD Ryzen 7 7800X3D 8-Core Processor


In [3]:
DEVICE="cuda:0"

In [4]:
DATASET_DIR = "../dataset"

ORIG_FILE = "./output/original.wav"
SPEAKER1_FILE = "./output/speaker1.wav"
SPEAKER2_FILE = "./output/speaker2.wav"
SPEAKER1_METRICS_FILE = "./output/speaker1.json"
SPEAKER2_METRICS_FILE = "./output/speaker2.json"

SPEAKER1_DIR = "./output/speaker1"
SPEAKER2_DIR = "./output/speaker2"

MIN_SEGMENT_LENGTH_SEC = 0.1
STOP_PHRASES = [
    "ДИНАМИЧНАЯ МУЗЫКА",
    "Продолжение следует.",
    "Продолжение следует...",
]
STOP_PHRASE_LENGTH_DELTA = 5

PER_PHRASE_METRICS = []
PER_PHRASE_METRIC_NAMES = ["whisper"]
PER_RECORDING_METRICS = []
PER_RECORDING_METRIC_NAMES = []

In [5]:
def per_phrase(func):
    PER_PHRASE_METRICS.append(func)
    PER_PHRASE_METRIC_NAMES.append(func.__name__)
    
def per_recording(func):
    PER_RECORDING_METRICS.append(func)
    PER_RECORDING_METRIC_NAMES.append(func.__name__)

In [6]:
import os

os.makedirs(SPEAKER1_DIR, exist_ok=True)
os.makedirs(SPEAKER2_DIR, exist_ok=True)

In [7]:
import random

input_file = random.choice(os.listdir(DATASET_DIR))
input_file = f"{DATASET_DIR}/" + input_file
input_file

'../dataset/mix_13053_16e012__2025_10_01__09_25_43_960.mp3'

In [8]:
import pydub

audio = pydub.AudioSegment.from_file(input_file)

try:
    left_channel, right_channel = audio.split_to_mono()
except ValueError as ex:
    raise ValueError("Input file must be stereo.") from ex

left_channel.export(SPEAKER1_FILE, format="wav")
right_channel.export(SPEAKER2_FILE, format="wav")
audio.export(ORIG_FILE, format="wav")

<_io.BufferedRandom name='./output/original.wav'>

In [9]:
import whisper
whispermodel = whisper.load_model("large").to(DEVICE)

In [10]:
from typing import Generator, Tuple

def filter_out(segment_data: dict) -> bool:
    # Removing short segments
    if (segment_data["end"] - segment_data["start"]) < MIN_SEGMENT_LENGTH_SEC:
        return True

    # Removing segments that contain stop_phrases
    matching_stop_phrases = [s for s in STOP_PHRASES if s.lower() in segment_data["text"].lower()]
    if matching_stop_phrases:
        # Remove the segment if it contains only a stop phrase
        if max(map(len, matching_stop_phrases)) + STOP_PHRASE_LENGTH_DELTA > len(segment_data["text"].strip()):
            return True
            
    return False

def segmentize(source: str, segments_dir: str) -> Generator[Tuple[dict, str]]:
    transcription = whispermodel.transcribe(source, word_timestamps=True, language='ru')
    audio = pydub.AudioSegment.from_file(source)
    for segment_data in transcription['segments']:
        if filter_out(segment_data):
            continue
        start = float(segment_data['start']) * 1000
        end = float(segment_data['end']) * 1000
        path = f"{segments_dir}/{segment_data['id']}.wav"
        audio[start:end].export(path, format="wav")
        yield (segment_data, path)

In [11]:
def show(vals: dict) -> dict:
    keys = ["id", "start", "end", "text"]
    return {k: vals[0][k] for k in keys}
list(map(show, segmentize(SPEAKER1_FILE, SPEAKER1_DIR)))

[{'id': 0,
  'start': np.float64(0.0),
  'end': np.float64(6.34),
  'text': ' Добрый день, меня зовут Вадим, я представляю Автомир, официального дилера ХВЛ.'},
 {'id': 1,
  'start': np.float64(6.54),
  'end': np.float64(10.16),
  'text': ' Вы планируете покупку нового автомобиля ХВЛ в Самаре в ближайший месяц?'},
 {'id': 2,
  'start': np.float64(15.08),
  'end': np.float64(17.28),
  'text': ' Скажите, какая модель вас интересует?'},
 {'id': 3,
  'start': np.float64(22.06),
  'end': np.float64(23.68),
  'text': ' F7 или H7?'},
 {'id': 4,
  'start': np.float64(25.62),
  'end': np.float64(26.42),
  'text': ' F7?'},
 {'id': 5,
  'start': np.float64(28.860000000000003),
  'end': np.float64(32.04),
  'text': ' А покупку планируете как физическое или как юридическое лицо?'},
 {'id': 6,
  'start': np.float64(36.06),
  'end': np.float64(40.88),
  'text': ' Тогда давайте я прямо сейчас соединю вас с менеджером, который подберет предложение по вашим параметрам.'},
 {'id': 7,
  'start': np.float64

In [12]:
from typing import Callable, Any

def collect_metrics_for_segment(
    source: str,
    phrase_model_callbacks: list[Callable[[str], dict[str, Any]]],
    recording_model_callbacks: list[Callable[[str], dict[str, Any]]],
) -> dict[str, dict]:
    return (
        {model.__name__: model(source) for model in phrase_model_callbacks},
        {model.__name__: model(source) for model in recording_model_callbacks},
    )

def collect_metrics(
    source: str,
    segments_dir: str,
    phrase_model_callbacks: list[Callable[[str], dict[str, Any]]],
    recording_model_callbacks: list[Callable[[str], dict[str, Any]]],
) -> Generator[dict[str, dict[str, Any]]]:
    for segment_data, x in segmentize(source, segments_dir):
        text = segment_data["text"]
        print(segment_data["id"], f"[{text}]")
        (per_phrase, per_recording) = collect_metrics_for_segment(x, phrase_model_callbacks, recording_model_callbacks)
        yield (per_phrase | dict(whisper=segment_data), per_recording)

In [13]:
from funasr import AutoModel

# model="iic/emotion2vec_base"
# model="iic/emotion2vec_base_finetuned"
# model="iic/emotion2vec_plus_seed"
# model="iic/emotion2vec_plus_base"
model_id = "iic/emotion2vec_plus_large"

emomodel = AutoModel(
    model=model_id,
    hub="hf",
    disable_update=True,
    device=DEVICE,
    log_level="CRITICAL",
)

_ = emomodel.model.to(DEVICE)

funasr version: 1.2.7.


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.0.0.weight, /home/gmatiukhin/.cache/huggingface/hub/models--emotion2vec--emotion2vec_plus_large/snapshots/6c303ba987b86b93193de93e34bb2b077a6bedc4/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.0.0.bias, /home/gmatiukhin/.cache/huggingface/hub/models--emotion2vec--emotion2vec_plus_large/snapshots/6c303ba987b86b93193de93e34bb2b077a6bedc4/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.1.0.weight, /home/gmatiukhin/.cache/huggingface/hub/models--emotion2vec--emotion2vec_plus_large/snapshots/6c303ba987b86b93193de93e34bb2b077a6bedc4/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.1.0.bias, /home/gmatiukhin/.cache/huggingface/hub/models--emotion2vec--emotion2vec_plus_large/snapshots/6c303ba987b86b93193de93e34bb2b077a6bedc4/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.2.0.weight, /home/gmatiukhin/.cache/huggingface/hu

In [14]:
@per_phrase
def emotion2vec(segment_path: str) -> dict[str, Any]:
    out = emomodel.generate(segment_path, granularity="utterance", extract_embedding=False)
    return {k.split("/")[-1]: v for (k, v) in zip(out[0]["labels"], out[0]["scores"])}

In [15]:
from analysis_node.analysis import EmotionPipeline

emotion_pipeline = EmotionPipeline(DEVICE)

In [16]:
from scipy.io import wavfile

@per_phrase
def wav2vec2_emotion(segment_file: str) -> dict[str, Any]:
    sample_rate, waveform = wavfile.read(segment_file)
    vals = emotion_pipeline(waveform, sample_rate)
    return {k: v for (k, v) in zip(["arousal", "dominance", "valence"], vals[0])}

In [17]:
from analysis_node.analysis import AgeGenderPipeline

age_gender_pipeline = AgeGenderPipeline("small", DEVICE)

In [18]:
@per_recording
def wav2vec2_age_gender(segment_file: str) -> dict[str, Any]:
    sample_rate, waveform = wavfile.read(segment_file)
    vals = age_gender_pipeline(waveform, sample_rate)
    out = {k: v for (k, v) in zip(["age", "female", "male", "child"], vals[0])}
    return out

In [19]:
raw_metrics = list(collect_metrics(SPEAKER1_FILE, SPEAKER1_DIR, PER_PHRASE_METRICS, PER_RECORDING_METRICS))
raw_metrics

0 [ Добрый день, меня зовут Вадим, я представляю Автомир, официального дилера ХВЛ.]


rtf_avg: 0.018: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.48it/s]


1 [ Вы планируете покупку нового автомобиля ХВЛ в Самаре в ближайший месяц?]


rtf_avg: 0.016: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.10it/s]


2 [ Скажите, какая модель вас интересует?]


...

10 [ А хорошо, тогда не кладите трубку, прямо сейчас соединяюсь.]


rtf_avg: 0.267: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.36it/s]


11 [ Спасибо, менеджер.]


rtf_avg: 0.086: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 31.47it/s]


[({'emotion2vec': {'angry': 3.7985137169016525e-05,
    'disgusted': 0.00010294616367900744,
    'fearful': 0.0001235693198395893,
    'happy': 0.002397349802777171,
    'neutral': 0.9833465814590454,
    'other': 8.582897862652317e-05,
    'sad': 0.0006209969869814813,
    'surprised': 1.0889703844441101e-05,
    '<unk>': 0.013273905962705612},
   'wav2vec2_emotion': {'arousal': np.float32(0.7356798),
    'dominance': np.float32(0.6685522),
    'valence': np.float32(0.3694641)},
   'whisper': {'id': 0,
    'seek': 0,
    'start': np.float64(0.0),
    'end': np.float64(6.34),
    'text': ' Добрый день, меня зовут Вадим, я представляю Автомир, официального дилера ХВЛ.',
    'tokens': [50365,
     3401,
     13829,
     4851,
     13509,
     11,
     6885,
     46376,
     2348,
     2601,
     2165,
     11,
     2552,
     39412,
     1148,
     50175,
     17804,
     4490,
     11,
     31950,
     30321,
     31227,
     1070,
     2338,
     13126,
     9456,
     8578,
     14854

In [20]:
import numpy as np

phrase_metrics, recording_metrics = zip(*raw_metrics)

metrics = dict(
    recording = dict(),
    phrase = list(phrase_metrics)
)

for name in PER_RECORDING_METRIC_NAMES:
    mean = np.mean([list(metrics[name].values()) for metrics in recording_metrics], axis=0)
    metrics["recording"][name] = {k: v for (k, v) in zip(recording_metrics[0][name].keys(), mean)}

metrics

{'recording': {'wav2vec2_age_gender': {'age': np.float32(0.2541189),
   'female': np.float32(0.73997617),
   'male': np.float32(0.02599925),
   'child': np.float32(0.23402464)}},
 'phrase': [{'emotion2vec': {'angry': 3.7985137169016525e-05,
    'disgusted': 0.00010294616367900744,
    'fearful': 0.0001235693198395893,
    'happy': 0.002397349802777171,
    'neutral': 0.9833465814590454,
    'other': 8.582897862652317e-05,
    'sad': 0.0006209969869814813,
    'surprised': 1.0889703844441101e-05,
    '<unk>': 0.013273905962705612},
   'wav2vec2_emotion': {'arousal': np.float32(0.7356798),
    'dominance': np.float32(0.6685522),
    'valence': np.float32(0.3694641)},
   'whisper': {'id': 0,
    'seek': 0,
    'start': np.float64(0.0),
    'end': np.float64(6.34),
    'text': ' Добрый день, меня зовут Вадим, я представляю Автомир, официального дилера ХВЛ.',
    'tokens': [50365,
     3401,
     13829,
     4851,
     13509,
     11,
     6885,
     46376,
     2348,
     2601,
     2165,


In [21]:
import json

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

json_str = json.dumps(metrics, indent=4, cls=NpEncoder)
with open(SPEAKER1_METRICS_FILE, "w") as f:
    f.write(json_str)